In [1]:
import pandas as pd
import numpy as np
import random
import scipy.stats as stats
from sklearn.ensemble import GradientBoostingRegressor
from sklearn import cross_validation, metrics

Steps:
1) Initial population (20) chromosomes:
    - 3 parts: [n-estimators][learning-rate][features]
               [600-1200]    [0.01-0.07]    [0,1 presence]
    - n-estimators: randint(0, 30)*20+600
    - learning-rate: randint(1, 5)*0.01
    - features: if random() > 0.7, include feature
2) Determine fitness:
    - feed values to gbr, get score
        - for n-estimators, round to nearest 10
    - fitness = wa * accuracy + wf * (1/sum(feature-presence))
        - wa, wf weights, wf meant to penalize incorporating large number of features
        - probably wa ~95-100 (num-features does not matter)
3) Hold out "elite" (top ~10%)
4) Crossover
    - repeat while len(newPop) < len(oldPop):
    - select of 2 parents: select random number r between 0 and sum of pop's fitness
                           sum through population until reach a number > r 
    - choose random crossover point, switch all after that
5) Mutation
    - for each bit, if random() < 0.001, flip


Overall:
Generate population
for x iterations:
    * evolve population (grade fitness, hold out, crossover, mutation -> return newPop)
return fittest

In [2]:
# orig_data = pd.DataFrame.from_csv('Clean_data/train.csv', header=0, index_col=False)
df = pd.DataFrame.from_csv('Train_data/with_interaction_biogrid.csv')
df.columns

Index(['CELL_LINE', 'COMPOUND_A', 'COMPOUND_B', 'MAX_CONC_A', 'MAX_CONC_B',
       'IC50_A', 'H_A', 'Einf_A', 'IC50_B', 'H_B', 'Einf_B', 'SYNERGY_SCORE',
       'A_HBA', 'A_cLogP', 'A_HBD', 'A_Lipinski', 'A_MW', 'A_ALogP/XLogP',
       'A_PSA', 'A_#RotBonds', 'A_Arom Rings', 'A_Heavy atoms', 'A_QED',
       'B_HBA', 'B_cLogP', 'B_HBD', 'B_Lipinski', 'B_MW', 'B_ALogP/XLogP',
       'B_PSA', 'B_#RotBonds', 'B_Arom Rings', 'B_Heavy atoms', 'B_QED',
       'GEX_CLUSTER', 'THERAPY_CLUSTER', 'Disease_area', 'Target_share',
       'Target_interaction'],
      dtype='object')

In [3]:
df.columns[23]

'B_HBA'

In [4]:
num_features = 11 # just added features

#### feature chromosome represents: columns[11-21] -> [22-32]

In [5]:
# returns binary string of sepcified length
def toBinary(x, length):
    bstring = bin(x)[2:]
    return (length-len(bstring))*'0'+bstring

def generatePop(count):
    pop = []
    for i in range(count):
        c1 = toBinary(random.randint(0, 30), 5)
        c2 = toBinary(random.randint(1, 7), 3)
        c3arr = [int(random.random()>0.5) for x in range(num_features)]
        c3 = ''.join(str(x) for x in c3arr)
        chromo = c1+c2+c3
        pop.append(chromo)
    return pop

In [6]:
print(generatePop(5))

['1001001011010111010', '1011101101011101101', '0010011110110110111', '0110011000000110101', '0111101001011000110']


In [7]:
def fitness(ind, wa):
    c1 = ind[:5]
    c2 = ind[5:8]
    c3 = ind[8:]
    num_est = int(c1, 2)*20+600
    lr = int(c2, 2)*0.01
    feats = [int(x) for x in list(c3)]
    
    # drop features whose corresponding position is 0:
    toDrop = []
    for i in range(len(feats)):
        if feats[i]==0:
            toDrop.append(df.columns[i+11]) # A_features
            toDrop.append(df.columns[i+22]) # B_features
    df_mod = df.drop(toDrop, axis=1)
    
    # build up X, y train/test values:
    df_mod = df_mod.iloc[np.random.permutation(len(orig_data))]
    Xpd = pd.get_dummies(df_mod, columns=['CELL_LINE', 'COMPOUND_A', 'COMPOUND_B'])
    Ypd = orig_data['SYNERGY_SCORE'].reindex(df_mod.index)
    X_train, X_test, y_train, y_test = cross_validation.train_test_split(Xpd.values, Ypd.values)
    
    
    # train model
    gbr = GradientBoostingRegressor(n_estimators=num_est, learning_rate=lr, 
                                    max_depth=7, max_features='log2')
    gbr.fit(X_train, y_train)
    #accuracy = gbr.score(X_test, y_test)
    accuracy = stats.pearsonr(y_test, gbr.predict(X_test))[0]
    
    # calculate fitness score: wa (weight accuracy) vs wf (weight features = inverse size)
    fitness = wa*accuracy + (1-wa)*(1/sum(feats))
    return fitness

In [8]:
def select(pop):
    fitnesses = [fitness(ind, 0.9) for ind in pop]
    r = random.random()*sum(fitnesses)
    count = 0
    for i in range(len(pop)):
        count += fitnesses[i]
        if count > r:
            return pop[i]

def crossover(p1, p2):
    while True:
        crossSite = random.randint(1, len(p1)-2)
        c1 = p1[:crossSite] + p2[crossSite:]
        c2 = p2[:crossSite] + p1[crossSite:]
        if '1' in c1[5:8] and '1' in c2[5:8]: # if learning rate not 0
            break
    return c1, c2

def mutate(p1, muteRate):
    mutated = ''
    for bit in p1:
        if random.random() < muteRate:
            mutated += ('1' if (bit=='0') else '0')
        else:
            mutated += bit
    return mutated

In [15]:
def evolve(pop, muteRate=0.001):
    # "Hold-out" elite
#     scores = [(fitness(ind, 0.95), ind) for ind in pop] # pairs of (fitness, individual)
#     ranked = [x[1] for x in sorted(scores)] # ranked individuals by fitness
#     retain_pos = int(0.1*len(ranked))
#     newPop = ranked[:retain_pos]
    
    newPop = []
    # Populate with children
    while (len(newPop) < len(pop)):
        p1, p2 = select(pop), select(pop)
        c1, c2 = crossover(p1, p2)
        c1, c2 = mutate(c1, muteRate), mutate(c2, muteRate)
        newPop.extend([c1, c2])
    return newPop

In [16]:
def bestFit(pop):
    scores = [(fitness(ind, 0.95), ind) for ind in pop] # pairs of (fitness, individual)
    best = sorted(scores)[0]
    return best[1], best[0] # individual, fitness

In [17]:
p = generatePop(20)
print(str(bestFit(p)[1]) + ' -> ' + str(bestFit(p)[0]))
for i in range(30):
    p = evolve(p)
    best = bestFit(p)
    print(str(best[1]) + ' -> ' + str(best[0]))

0.461508394162 -> 1010110001001100011
0.499778074272 -> 0001010110001101001
0.473063095768 -> 1110001100101111101
0.492666151857 -> 0110010111001111101
0.508008473113 -> 0100111111101111101
0.473484149035 -> 0110001101101110000
0.458874106813 -> 1100111000101001101
0.516382484706 -> 0011101010101111110
0.460321168599 -> 1100111000101111101
0.483271194632 -> 0011101010101111101
0.513440513924 -> 0011101001101111101
0.472534094187 -> 0011101000101001101
0.469063623211 -> 0011111010101001101
0.470730712203 -> 0001101010101110001
0.511438061817 -> 0001101011101001101
0.486452898494 -> 0011101011101110001
0.467405514848 -> 0011111010101111101
0.482687502074 -> 0011101000101111101
0.451012207507 -> 0011101011101110001
0.503846301852 -> 0011101000101110101
0.492362507573 -> 0011101000101110001
0.504055325968 -> 0011101010101110001
0.458718125847 -> 0011101011101110001
0.515953951759 -> 0011101010101110101
0.484393997559 -> 1001101010101110101
0.463195747374 -> 0011101010101110001
0.5015545523

0.517641793998
0.437962584177
0.452752921057
0.477713745149
0.512090991349
0.499979297675
0.487268919544
0.46369458358
0.487436052318
0.521185332243
0.469072111826
0.49661530266
0.518260209555
0.505441707497
0.511000291084
0.503450823409

0.461508394162 -> 1010110001001100011
0.499778074272 -> 0001010110001101001
0.473063095768 -> 1110001100101111101
0.492666151857 -> 0110010111001111101
0.508008473113 -> 0100111111101111101
0.473484149035 -> 0110001101101110000
0.458874106813 -> 1100111000101001101
0.516382484706 -> 0011101010101111110
0.460321168599 -> 1100111000101111101
0.483271194632 -> 0011101010101111101
0.513440513924 -> 0011101001101111101
0.472534094187 -> 0011101000101001101
0.469063623211 -> 0011111010101001101
0.470730712203 -> 0001101010101110001
0.511438061817 -> 0001101011101001101
0.486452898494 -> 0011101011101110001
0.467405514848 -> 0011111010101111101
0.482687502074 -> 0011101000101111101
0.451012207507 -> 0011101011101110001
0.503846301852 -> 0011101000101110101
0.492362507573 -> 0011101000101110001
0.504055325968 -> 0011101010101110001
0.458718125847 -> 0011101011101110001
0.515953951759 -> 0011101010101110101
0.484393997559 -> 1001101010101110101
0.463195747374 -> 0011101010101110001
0.501554552372 -> 0011101010101110101
0.504565220082 -> 0011101010101110101
0.505240131371 -> 0011101010101110001
0.515311665303 -> 1011101010101110001
0.483897311763 -> 0001101010101110001

0.517641793998
0.437962584177
0.452752921057
0.477713745149
0.512090991349
0.499979297675
0.487268919544
0.46369458358
0.487436052318
0.521185332243
0.469072111826
0.49661530266
0.518260209555
0.505441707497
0.511000291084
0.503450823409

### Feature selection only

In [25]:
def generatePop(count):
    pop = []
    for i in range(count):
        feats = [int(random.random()>0.35) for x in range(num_features)]
        pop.append(feats)
    return pop

In [26]:
generatePop(5)

[[0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1],
 [1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1],
 [1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1],
 [0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0],
 [1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1]]

In [7]:
def pearson(y, y_pred):
    return stats.pearsonr(y, y_pred)[0]
pearson_scorer = metrics.make_scorer(pearson)

In [9]:
def fitness(ind, wa):
    # drop features whose corresponding position is 0:
    toDrop = []
    for i in range(len(ind)):
        if ind[i]==0:
            toDrop.append(df.columns[i+12]) # A_features
            toDrop.append(df.columns[i+23]) # B_features
    dfmod = df.drop(toDrop, axis=1)
    
    # build up X, y train/test values:
    dfmod = dfmod.iloc[np.random.permutation(len(dfmod))]
    y = dfmod['SYNERGY_SCORE'].values
    X = pd.get_dummies(dfmod.drop('SYNERGY_SCORE', axis=1), columns=['CELL_LINE', 'COMPOUND_A', 'COMPOUND_B', 'Disease_area']).values

    # train model
    gbr = GradientBoostingRegressor(n_estimators=1000, max_depth=5, loss='huber',
                                    max_features='log2', learning_rate=0.03)
    pearsons = cross_validation.cross_val_score(gbr, X, y, scoring=pearson_scorer, cv=3)
    accuracy = np.mean(pearsons)
    
    # calculate fitness score: wa (weight accuracy) vs wf (weight features = inverse size)
    fitness = wa*accuracy + (1-wa)*(1/sum(ind))
    return fitness

In [22]:
def select(pop):
    fitnesses = [fitness(ind, 1) for ind in pop] # for now, full weight to accuracy
    r = random.random()*sum(fitnesses)
    count = 0
    for i in range(len(pop)):
        count += fitnesses[i]
        if count > r:
            return pop[i]

def crossover(p1, p2):
    crossSite = random.randint(0, len(p1))
    c1 = p1[:crossSite] + p2[crossSite:]
    c2 = p2[:crossSite] + p1[crossSite:]
    return c1, c2

def mutate(p1, muteRate):
    mutated = []
    for bit in p1:
        if random.random() < muteRate:
            mutated.append(1 if (bit==0) else 0)
        else:
            mutated.append(bit)
    return mutated

In [23]:
def evolve(pop, muteRate=0.002):
    # "Hold-out" elite
    scores = [(fitness(ind, 1), ind) for ind in pop] # pairs of (fitness, individual)
    ranked = [x[1] for x in sorted(scores)] # ranked individuals by fitness
    retain_pos = int(0.1*len(ranked))
    newPop = ranked[:retain_pos]
    
    # Populate with children
    while (len(newPop) < len(pop)):
        p1, p2 = select(pop), select(pop)
        c1, c2 = crossover(p1, p2)
        c1, c2 = mutate(c1, muteRate), mutate(c2, muteRate)
        newPop.extend([c1, c2])
    return newPop

In [24]:
def bestFit(pop):
    scores = [(fitness(ind, 1), ind) for ind in pop] # pairs of (fitness, individual)
    best = sorted(scores)[0]
    return best[1], best[0] # individual, fitness

In [27]:
p = generatePop(8)
print(str(bestFit(p)[1]) + ' <- ' + str(bestFit(p)[0]))
for i in range(20):
    p = evolve(p)
    best = bestFit(p)
    print(str(best[1]) + ' <- ' + str(best[0]))

0.596177766606 <- [1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1]
0.590212587455 <- [1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1]
0.605973428593 <- [1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1]
0.589519542154 <- [1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1]
0.562457935754 <- [1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1]
0.588368359399 <- [1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1]
0.584591435057 <- [1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1]
0.560480306308 <- [1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1]
0.565791667345 <- [1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1]
0.539392647747 <- [1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1]
0.588772582758 <- [1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1]
0.58418857158 <- [1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1]
0.569714309631 <- [1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1]
0.583231324348 <- [1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1]
0.606731052479 <- [1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1]
0.577395816537 <- [1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1]
0.589689538235 <- [1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1]
0.60019046983 <- [1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1]
0.601716715333 <- [1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1]
0.580162107548